## Assignment 6 - Path Analysis



In [53]:
# imports
import numpy as np
import itertools as it
from scipy.stats import pearsonr

### Problem 1 
Program a function that given a causal graph computes the basis set of independences to be checked to assert if the graph is a feasible causal structure.

The basis set of independeces is basically the minimum number of d-separation statements, so we can just program the method that was proposed by Pearl.
Pearls basis set contains d-separation statements consisting of each such pair of vertices conditioned on the parents of the vertex having a higher causal order.

1. To obtain the basis set we first list each unique pair of non-adjacent vertices.
2. Then we list all causal parents of each vertex in the pair 

In [185]:
# input for causalGraph is a matrix
def getBasisSet (causalGraph):
    # add all pairs between vertices to the list
    basisSet = (list(it.combinations(range(len(causalGraph)),2)))
    print(basisSet)
    # remove pairs that are adjacent
    for n in range(len(causalGraph)):
        for m in range(len(causalGraph)):
            if (causalGraph[n,m] == 1):
                basisSet = [basisSet[i] for i in range(len(basisSet)) 
                                     if not(basisSet[i] == (n,m) 
                                            or basisSet[i] == (m,n))]
    # add causal parents to each vertex in the pair 
    # the first two elements in the tuples represent the vertices, and the rest in the tuple are the parents 
    for i in range(len(basisSet)):
        # print(basisSet[i])
        for k in range(len(causalGraph)):
            if causalGraph[k, basisSet[i][0]] == 1 and not k in basisSet[i]:
                basisSet[i] = basisSet[i] + (k,)
            if causalGraph[k, basisSet[i][1]] == 1 and not k in basisSet[i]:
                basisSet[i] = basisSet[i] + (k,)
                
    print(basisSet)
    return basisSet

In [186]:
# Generate testing data (nxn-matrix with 0's and 1's) for the function that is a causal graph

#x0 -> x1, x2->x1, x2->x3
basisTest = getBasisSet(np.array([[0, 1, 0, 0], 
                                  [0, 0, 0, 0], 
                                  [0, 1, 0, 1], 
                                  [0, 0, 0, 0]]))

#x0 -> x1, x2->x1 , x3->x2
basisTest2 = getBasisSet(np.array([[0, 1, 0, 0], 
                                  [0, 0, 0, 0], 
                                  [0, 1, 0, 0], 
                                  [0, 0, 1, 0]]))

[(0, 1), (0, 2), (0, 3), (1, 2), (1, 3), (2, 3)]
[(0, 2), (0, 3, 2), (1, 3, 0, 2)]
[(0, 1), (0, 2), (0, 3), (1, 2), (1, 3), (2, 3)]
[(0, 2, 3), (0, 3), (1, 3, 0, 2)]


The function getBasisSet() returns the basisSet for the given causalGraph as input. The output is a list of d-separation statements. Each d-separation statement is a tuple of which the first two elements represent the vertices and the following elements represent the causal parents to each vertex.

## Problem 2

Program a function that given a basis set and data calculates the independences of the given data (with Pearson Correlation) and returns True if the independeces are observed.

We assume that in each pair of verteces both are normally distributed and any relationship between them is linear so we can use Pearson Correlation to measure the independence.

We basically want to check if the given data follows the causal model with the given basisSet and if all vertices are independent the data follows the model.
How could data look like? The data could be a matrix with normal-distributed values for each variable (x0 x1 x2 x3...)
For checking the d-separation statements we have to condition on certain variables, we can achieve that by taking subsets of the data, for which the variable that should be conditioned has a fixed value each. Then we take the weighted average of the correlations of each subset.

Normal-distributed values are continous. To be able to find subsets of the data with a fixed value for each conditioned variable i'm going to round the data and like that put the values into bins.

In some cases we just have subsets of data that contain only one row of the data. In that case we cannot calculate the correlation so we are skipping this row.

To be able to condition on multiple variables at the same time i'm using recursion - it divides the data into subsets until it reaches the last variable that should be conditioned on. Then it calculates the correlation.

In [169]:
# function receives a basisSet and data as input and checks if the d-separation statements given in basisSet are fulfilled
# by the data, if yes function returns true otherwise false

weightedSumCorr = 0
numWeights = 0

def isIndependentForBasisSet (basisSet, data, threshold):
    global weightedSumCorr, numWeights
    
    # checks for each d-sep statement in the basisset if the correlation is under a certain threshold
    for statement in basisSet:
        weightedSumCorr = 0
        numWeights = 0
        i = 1
        # recursive function call to separate data into subsets and calculate correlation
        calculateCorr(statement, data, i)
                
        # if weightedCorr is over the threshold for correlation -> return False, otherwise go on
        weightedCorr = weightedSumCorr/numWeights
        print("Weighted correlation is: ", weightedCorr)
        
        if weightedCorr > threshold:
            # the correlation is over the threshold so the variables seem to be dependent
             return False
        
    # all variables' correlations were under the threshold, we consider the data to follow the given basisSet 
    return True


def calculateCorr(statement, data, i):
    global weightedSumCorr, numWeights
    
    # end-condition, correlation in the last variable that has to be conditioned
    if i == len(statement)-1: 
            # calculate the correlation
            # insert the case that we have less than two values (what do i do then? -> skipping it)
            if len(data) > 1:
                corr = pearsonr(data[:,statement[0]],data[:,statement[1]])
                print(abs(corr[0]))
                # and calculate the weightedSum and the number of Weights
                weightedSumCorr = weightedSumCorr + abs(corr[0]) * len(data)
                numWeights = numWeights + len(data)
    else:
        # find unique values for the variable to condition on 
            allValues = np.unique(data[:,statement[i+1]])
            print("Unique values are: ", allValues)
            # find for each value the rows of the data that have to be used for calculating the correlation
            for value in allValues:
                print("corrArr, value of conditioned variable:", value, "conditioned variable", statement[i+1])
                corrArr = data[np.where(data[:,statement[i+1]] == value)]
                # call recursive function with subsets of data
                print("CorrArray:",corrArr)
                print("Correlation between:", corrArr[:,statement[0]], " and: ", corrArr[:,statement[1]]) 
                calculateCorr(statement, corrArr, i+1)
                  
        

In [190]:
# Generate testing data for the first basisSet (causalGraph of exercise 1)
# I need the columns of the np-array to be normally distributed data and then i round it to put it into bins
mu1, sigma1 = 50, 3
mu2, sigma2 = 50 , 3
mu3, sigma3 = 50 , 3
mu4, sigma4 = 50, 3
# to test data that should follow the causal graph i'm adding to each variable the parent variable to relate them
# the result should be true because the variables are unrelated apart from the causal relationships
var0 = np.rint(np.random.normal(mu1, sigma1, 500))
var1 = var0 + var2 + np.rint(np.random.normal(mu2, sigma2, 500))
var2 = np.rint(np.random.normal(mu3, sigma3, 500))
var3 = var2 + np.rint(np.random.normal(mu4, sigma4, 500))
test = np.rint(np.column_stack((var0, var1, var2, var3)))
print(test)

isIndependentForBasisSet (basisTest, test, 0.3)

[[ 48. 205.  45.  91.]
 [ 50. 210.  54. 102.]
 [ 49. 188.  52. 106.]
 ...
 [ 51. 187.  47.  95.]
 [ 48. 183.  51. 102.]
 [ 54. 198.  48.  93.]]
0.09185723346176822
Weighted correlation is:  0.09185723346176822
Unique values are:  [41. 42. 43. 44. 45. 46. 47. 48. 49. 50. 51. 52. 53. 54. 55. 56. 57. 58.]
corrArr, value of conditioned variable: 41.0 conditioned variable 2
CorrArray: [[ 48. 210.  41.  96.]]
Correlation between: [48.]  and:  [96.]
corrArr, value of conditioned variable: 42.0 conditioned variable 2
CorrArray: [[ 51. 197.  42.  91.]
 [ 45. 201.  42.  89.]]
Correlation between: [51. 45.]  and:  [91. 89.]
1.0
corrArr, value of conditioned variable: 43.0 conditioned variable 2
CorrArray: [[ 46. 212.  43.  94.]
 [ 50. 206.  43.  94.]
 [ 51. 201.  43.  93.]
 [ 50. 206.  43.  93.]]
Correlation between: [46. 50. 51. 50.]  and:  [94. 94. 93. 93.]
0.6509445549041194
corrArr, value of conditioned variable: 44.0 conditioned variable 2
CorrArray: [[ 51. 190.  44.  94.]
 [ 50. 203.  44.  

  99. 108. 105. 103. 102.]
0.036268497497534136
corrArr, value of conditioned variable: 55.0 conditioned variable 2
CorrArray: [[ 52. 206.  55. 103.]
 [ 47. 196.  55. 106.]
 [ 45. 188.  55. 107.]
 [ 44. 186.  55. 104.]
 [ 52. 206.  55. 111.]
 [ 45. 193.  55. 105.]
 [ 47. 197.  55. 107.]
 [ 54. 199.  55. 104.]
 [ 54. 196.  55. 106.]
 [ 49. 201.  55. 102.]
 [ 55. 214.  55. 108.]
 [ 51. 204.  55. 105.]
 [ 44. 194.  55. 103.]
 [ 49. 207.  55. 102.]
 [ 49. 198.  55. 104.]
 [ 48. 196.  55. 100.]
 [ 49. 194.  55. 104.]
 [ 47. 196.  55. 108.]
 [ 51. 214.  55. 105.]
 [ 51. 210.  55. 109.]
 [ 52. 207.  55. 104.]
 [ 60. 213.  55. 108.]
 [ 55. 207.  55. 105.]
 [ 49. 189.  55. 104.]
 [ 46. 204.  55. 103.]]
Correlation between: [52. 47. 45. 44. 52. 45. 47. 54. 54. 49. 55. 51. 44. 49. 49. 48. 49. 47.
 51. 51. 52. 60. 55. 49. 46.]  and:  [103. 106. 107. 104. 111. 105. 107. 104. 106. 102. 108. 105. 103. 102.
 104. 100. 104. 108. 105. 109. 104. 108. 105. 104. 103.]
0.30748492033611863
corrArr, value of 

0.5806513267897976
corrArr, value of conditioned variable: 47.0 conditioned variable 2
CorrArray: [[ 48. 202.  47.  96.]
 [ 48. 202.  47. 101.]
 [ 48. 197.  47.  97.]
 [ 48. 185.  47. 103.]
 [ 48. 205.  47.  98.]]
Correlation between: [202. 202. 197. 185. 205.]  and:  [ 96. 101.  97. 103.  98.]
0.6605818096460122
corrArr, value of conditioned variable: 48.0 conditioned variable 2
CorrArray: [[ 48. 200.  48.  95.]
 [ 48. 198.  48.  96.]
 [ 48. 200.  48.  99.]
 [ 48. 198.  48.  93.]
 [ 48. 198.  48. 104.]
 [ 48. 193.  48.  99.]
 [ 48. 196.  48.  99.]
 [ 48. 202.  48.  97.]
 [ 48. 205.  48.  94.]
 [ 48. 206.  48.  99.]]
Correlation between: [200. 198. 200. 198. 198. 193. 196. 202. 205. 206.]  and:  [ 95.  96.  99.  93. 104.  99.  99.  97.  94.  99.]
0.23692415692169547
corrArr, value of conditioned variable: 49.0 conditioned variable 2
CorrArray: [[ 48. 201.  49.  96.]
 [ 48. 201.  49. 104.]
 [ 48. 194.  49. 100.]]
Correlation between: [201. 201. 194.]  and:  [ 96. 104. 100.]
0.0
corrArr,

0.49963581494569725
corrArr, value of conditioned variable: 48.0 conditioned variable 2
CorrArray: [[ 51. 217.  48. 101.]
 [ 51. 206.  48. 100.]
 [ 51. 203.  48.  95.]
 [ 51. 203.  48.  96.]
 [ 51. 200.  48.  99.]
 [ 51. 201.  48. 101.]
 [ 51. 210.  48. 100.]
 [ 51. 190.  48.  96.]]
Correlation between: [217. 206. 203. 203. 200. 201. 210. 190.]  and:  [101. 100.  95.  96.  99. 101. 100.  96.]
0.5645449762265542
corrArr, value of conditioned variable: 49.0 conditioned variable 2
CorrArray: [[ 51. 199.  49.  99.]
 [ 51. 199.  49.  96.]
 [ 51. 211.  49. 102.]
 [ 51. 197.  49. 105.]]
Correlation between: [199. 199. 211. 197.]  and:  [ 99.  96. 102. 105.]
0.12097167578182677
corrArr, value of conditioned variable: 50.0 conditioned variable 2
CorrArray: [[ 51. 210.  50. 100.]
 [ 51. 206.  50. 103.]
 [ 51. 205.  50. 106.]
 [ 51. 208.  50.  98.]
 [ 51. 197.  50.  99.]
 [ 51. 205.  50. 101.]
 [ 51. 216.  50. 100.]
 [ 51. 202.  50.  99.]
 [ 51. 194.  50. 101.]
 [ 51. 208.  50.  98.]
 [ 51. 197. 

corrArr, value of conditioned variable: 57.0 conditioned variable 2
CorrArray: [[ 57. 194.  57. 107.]]
Correlation between: [194.]  and:  [107.]
corrArr, value of conditioned variable: 58.0 conditioned variable 0
CorrArray: [[ 58. 205.  51. 102.]]
Correlation between: [205.]  and:  [102.]
Unique values are:  [51.]
corrArr, value of conditioned variable: 51.0 conditioned variable 2
CorrArray: [[ 58. 205.  51. 102.]]
Correlation between: [205.]  and:  [102.]
corrArr, value of conditioned variable: 59.0 conditioned variable 0
CorrArray: [[ 59. 210.  53. 102.]
 [ 59. 212.  52. 100.]]
Correlation between: [210. 212.]  and:  [102. 100.]
Unique values are:  [52. 53.]
corrArr, value of conditioned variable: 52.0 conditioned variable 2
CorrArray: [[ 59. 212.  52. 100.]]
Correlation between: [212.]  and:  [100.]
corrArr, value of conditioned variable: 53.0 conditioned variable 2
CorrArray: [[ 59. 210.  53. 102.]]
Correlation between: [210.]  and:  [102.]
corrArr, value of conditioned variable: 6

True

In [189]:
# Generate testing data for the first basisSet (causalGraph of exercise 1)
# I need the columns of the np-array to be normally distributed data and then i round it to put it into bins
mu1, sigma1 = 50, 3
mu2, sigma2 = 50 , 3
mu3, sigma3 = 50 , 3
mu4, sigma4 = 50, 3
var0 = np.rint(np.random.normal(mu1, sigma1, 500))
var1 = np.rint(np.random.normal(mu2, sigma2, 500))
# variable 0 and 2 should be independent from each other --> check if it returns false!
var2 = 2*var0
var3 = np.rint(np.random.normal(mu4, sigma4, 500))
test2 = np.rint(np.column_stack((var0, var1, var2, var3)))
print(test2)
isIndependentForBasisSet (basisTest2, test2, 0.3)

[[ 52.  49. 104.  49.]
 [ 55.  49. 110.  48.]
 [ 43.  50.  86.  49.]
 ...
 [ 43.  50.  86.  52.]
 [ 45.  44.  90.  51.]
 [ 48.  53.  96.  49.]]
Unique values are:  [41. 42. 43. 44. 45. 46. 47. 48. 49. 50. 51. 52. 53. 54. 55. 56. 57. 58.
 59.]
corrArr, value of conditioned variable: 41.0 conditioned variable 3
CorrArray: [[ 50.  53. 100.  41.]]
Correlation between: [50.]  and:  [100.]
corrArr, value of conditioned variable: 42.0 conditioned variable 3
CorrArray: [[48. 54. 96. 42.]]
Correlation between: [48.]  and:  [96.]
corrArr, value of conditioned variable: 43.0 conditioned variable 3
CorrArray: [[ 52.  45. 104.  43.]
 [ 52.  45. 104.  43.]
 [ 52.  51. 104.  43.]
 [ 51.  48. 102.  43.]]
Correlation between: [52. 52. 52. 51.]  and:  [104. 104. 104. 102.]
1.0
corrArr, value of conditioned variable: 44.0 conditioned variable 3
CorrArray: [[ 48.  48.  96.  44.]
 [ 52.  53. 104.  44.]
 [ 55.  50. 110.  44.]
 [ 52.  52. 104.  44.]
 [ 52.  55. 104.  44.]
 [ 53.  50. 106.  44.]
 [ 45.  54.  

False